In [ ]:
# coding=utf-8
# Copyright 2023 Frank Latos AC8P
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#
# Much appreciation to the Pymoo project for providing the optimization framework used herein:
#
# pymoo: Multi-objective Optimization in Python
# https://github.com/anyoptimization/pymoo
# https://pymoo.org/index.html
#


# A Class for Representing Yagis and Other Antennas

This notebook contains a Python class for representing Yagis and other antennas constructed with telescoping tubing.
After the antenna is described declaratively (i.e. by setting various properties) the user can generate the NEC5 geometry cards for simulation, as well as simple solid models that can be displayed in the accompanying visualizer widget.

This is a first version -- probably contains bugs and/or inexplicable engineering misjudgements.

See the following notebooks for usage examples.


In [125]:

import numpy as np
from copy import deepcopy
from numba import njit
import plotly.graph_objects as go



def _convert(n, k):
    if isinstance(n,list):
        return np.array(n) * k
    return n * k

def in2m(inch):
    return _convert(inch, 1 / 39.3701)

def ft2m(ft):
    return _convert(ft, 12 / 39.3701)







class SolidObject():

    #
    # Translate and rotate antenna elements  (either a single array or list of arrays)
    #
    @staticmethod
    def _rot(elem,arr):
        if isinstance(elem, list):
            return [el @ arr for el in elem]
        return elem @ arr
    @staticmethod
    def rot_x(elem,ang):
        return SolidObject._rot(elem, np.array([[1,0,0],[0,np.cos(ang),np.sin(ang)],[0,-np.sin(ang),np.cos(ang)]]))
    @staticmethod
    def rot_y(elem,ang):
        return SolidObject._rot(elem, np.array([[np.cos(ang),0,-np.sin(ang)],[0,1,0],[np.sin(ang),0,np.cos(ang)]]))
    @staticmethod
    def rot_z(elem,ang):
        return SolidObject._rot(elem, np.array([[np.cos(ang),np.sin(ang),0],[-np.sin(ang),np.cos(ang),0],[0,0,1]]))
    @staticmethod
    def translate(elem, dxyz):
        rowarr = np.array([dxyz])
        if isinstance(elem, list):
            return [el + rowarr for el in elem]
        return elem + rowarr

    # Mirror list of arrays across xz plane
    @staticmethod
    def mirror_xz(arrs):
        mxz = np.array([[1,-1,1]])
        if isinstance(arrs, list):
            return [ar * mxz for ar in arrs]
        return arrs * mxz

    # Mirror list of arrays through origin
    @staticmethod
    def mirror_origin(arrs):
        mxz = np.array([[-1,-1,-1]])
        if isinstance(arrs, list):
            return [ar * mxz for ar in arrs]
        return arrs * mxz


    #
    # Create a polygon in xz plane, centered at (0,0,0)
    #
    #  nv       number of vertices
    #  r        distance from any vertex to origin
    #
    # Returns matrix of points (x,y,z)
    @staticmethod
    @njit
    def poly_xz(nv, r):
        assert nv > 2
        m = np.empty((nv,3))
        for i in range(nv):
            m[i] = np.array((r*np.cos(2*np.pi*i/nv), 0, r*np.sin(2*np.pi*i/nv)))
        return m

    #
    # Create a rectangle in xz plane, centered at (0,0,0)
    #
    #  x, z     width, height of rectangle
    #
    # Returns matrix of points (x,y,z)
    @staticmethod
    def rect_xz(x, z):
        return np.array([ [x/2,0,z/2],[x/2,0,-z/2],[-x/2,0,-z/2],[-x/2,0,z/2], ])

    #
    # Create a solid object by extruding a polygon along the y axis
    #
    #   m           array of points, shape (#vertices, 3)
    #   y           length of extrusion 
    #   close_end   close the first or second ends of extrusion?
    #   scale       scale of second face relative to first
    #
    # Returns: 
    #   s           array of points for both faces, shape (#vertices*2, 3)
    #   t           list of triangles, shape (#vertices*2, 3)
    #   
    @staticmethod
    def extrude_y(m, y, close_end=(False,False), scale=1.0):
        n = m.shape[0]                      # Number of points in initial shape
        s = np.vstack((m, m*scale + np.array((0,y,0)), np.array([[0,0,0],[0,y,0]])))     # Add other end face of solid
        t = []                              # Triangles (indexes of points in 's')
        for i in range(n):
            t.append( (i, i+n, (i+1)%n) )      # First triangle for this face
            t.append( ((i+1)%n, i+n, (i+1)%n + n) )     # Second triangle for this face
            if close_end[0]:
                t.append( (i, (i+1)%n, 2*n) )
            if close_end[1]:
                t.append( (i+n, (i+1)%n + n, 2*n+1) )
        return s, np.array(t)


    #
    # Create a cone
    #
    #   m           base of cone (array of points in xz plane)
    #   y           height (along y axis)
    #
    @staticmethod
    def cone_y(m, y):
        n = m.shape[0]                      # Number of points in initial shape
        s = np.vstack((m, np.array([[0,y,0]])))  
        t = np.empty((n, 3))              # Triangles (indexes of points in 's')
        for i in range(n):
            t[i] = np.array((i, (i+1)%n, n))      # Triangle for this face
        return s, t


    #
    # Return a list of one or more solid models for display  (override in derived classes)
    #
    # Returns: [ [point arrays], [triangle arrays], [css color strings], [hover text strings] ]
    #
    def get_solid_models(self):
        return [[],[],[],[]]
    




class SolidAntenna(SolidObject):

    def __init__(self, properties, n_elem):
        super().__init__()
        self.properties = {'tubing_dia':in2m([3/4, 5/8, 1/2]), 'display_dia':in2m([3/4, 5/8, 1/2]), 'section_lens':in2m([24,18]), 
                           'color':['lightblue','lightsteelblue','lightgray','whitesmoke'], 'nsides':9,
                           'boom':True, 'boomcolor':'steelblue', 'boomext':in2m(4), 'boomy':in2m(3), 'boomz':in2m(2),
                           'rotate':[], 'translate':(0,0,0)}
        self.properties.update(properties)
        self.els = [dict() for i in range(n_elem)]          # Elements (each represented as a dict)
        self.driven = 2     # Tag of driven element (can be overridden with 'driven' property on an element)

    # Use '[]' syntax to access the list of elements
    def __len__(self):
        return len(self.els)
    def __getitem__(self, key):             # Returns property dict for an element
        return self.els[key]
    def __setitem__(self,key,value):        # Assigns new properties
        self.els[key].update(value)


    # Get dict value (use global 'properties' if not present)
    def _getg(self, d, k, dv):
        return d[k] if (k in d) else self.properties.get(k,dv)


    # Return list of section lengths for this element dictionary
    def _sec_lens(self,d,el_idx):
        ellen = d['len']                                # Overall element length (must be present)
        seclens = self._getg(d,'section_lens',None)     # Fixed section lengths
        if seclens is None:                             # If not present, return single section length
            return [ellen]
        lens = np.hstack([seclens, ellen-np.sum(seclens)])      # seclens can be array or list...
        assert lens[-1] > 0, f'Error: Element[{el_idx}] length too short'
        return lens


    # Create element's NEC data structure:
    #  [ [2x3 array of points for each section], [radius for each section], tag#, [#segments for each section] ]
    def _make_el_nec_ds(self, d, el_idx):
        seclens = self._sec_lens(d, el_idx)             # Section lengths
        endpts = np.hstack([0.0,np.cumsum(seclens)])    # Section endpoints
        arrs = [ np.array([[0,endpts[i+0],0],[0,endpts[i+1],0]]) for i in range(len(endpts)-1) ]    # Endpoint arrays for each segment

        segnums = self._getg(d,'segnums',None)          # Segments per section
        if segnums is None:
            seg_per_m = self._getg(d,'seg_per_m',None)
            assert seg_per_m is not None, f"Error: Element[{el_idx}] 'segnums' or 'seg_per_m' must be specified"
            segnums = np.ceil(seclens * seg_per_m)
        segnums = np.array(segnums, dtype=int)
    
        tubing_dia = self._getg(d,'tubing_dia',None)
        assert tubing_dia is not None, f"Error: Element[{el_idx}] 'tubing_dia' must be specified"
        rads = np.array(tubing_dia) / 2

        tag = d.get('tag', el_idx+1)                    # Tag is either specified, or it's the element index plus one

        return [arrs, rads, tag, segnums]               # Return element data structure
    

    # Perform one or more rotations on a list of point arrays
    #  Rotations are specified like:  [ ('x', 45), ('y', 90), ('z', -90) ]
    #   arrs    list of point arrays
    #   rots    List of axes and rotations in degrees
    def _multi_rot(self, arrs, rots):
        for axis, degs in rots:                         # Axis and rotation angle pairs...
            if axis == 'x':
                arrs = SolidObject.rot_x(arrs, np.deg2rad(degs))
            if axis == 'y':
                arrs = SolidObject.rot_y(arrs, np.deg2rad(degs))
            if axis == 'z':
                arrs = SolidObject.rot_z(arrs, np.deg2rad(degs))
        return arrs


    #
    # Perform rotations, translation, mirroring of a NEC or solid-model data structure
    #   elds         an NEC or SM data struct
    #       --> these are lists, the first element of which is a list of arrays of points
    #         i.e. [ [one or more arrays of 3d points...], <other stuff specific to struct type...> ]
    #   rotate      List of rotations, e.g. [('x', 45), ('y', 90)], or None
    #   mirror      True to create mirrored element
    #   translate   Tuple of (x,y,z) offsets, or None
    #
    #   Returns: elds modified in place, mirrored element structure (if requested) as return value (else returns None)
    #
    def _rot_mir_trans(self, elds, rotate=None, mirror=False, translate=None):

        # First, rotations
        if rotate is not None:
            elds[0] = self._multi_rot(elds[0], rotate)

        # Next, mirror
        if mirror:
            mirror_elds = [SolidObject.mirror_xz(elds[0])] + elds[1:]     # Careful...not a deep copy...could use deepcopy()
        
        # Last, translate
        if translate is not None:
            elds[0] = SolidObject.translate(elds[0],translate)
            if mirror:
                mirror_elds[0] = SolidObject.translate(mirror_elds[0], translate)

        return mirror_elds if mirror else None
    

    #
    # Create a list of NEC data structures from element properties and global properties
    # Also sets driven-element tag if 'driven' property is present
    #
    # Returns:  a list of NEC data structs for elements, and possibly mirrored elements
    #
    def _create_nec_structs(self):
        dslist = []                                     # List of data structs to return
        for el_idx,d in enumerate(self.els):            # 'd' is the property dict for an element

            ds = self._make_el_nec_ds(d, el_idx)            # Create the nec data struct

            # Apply transforms from local properties dict
            rotate = d.get('rotate', None)              # Only local 'rotate' property
            mirror = self._getg(d,'nec_mirror',False)   # Local or global
            translate = d.get('translate', None)        # Only local 'translate' property
            # Perform transforms on ds, possibly return mirrored element
            mirror_ds = self._rot_mir_trans(ds, rotate, mirror, translate)
    
            # Next apply global rotate, translate (there's no global 'mirror')
            rotate = self.properties.get('rotate', None)  # Only global 'rotate' property
            translate = self.properties.get('translate', None)  # Only global 'translate' property
            self._rot_mir_trans(ds, rotate=rotate, translate=translate)
            dslist.append(ds)                           # Append to element list
            if mirror_ds is not None:
                self._rot_mir_trans(mirror_ds, rotate=rotate, translate=translate)
                dslist.append(mirror_ds)

            # Is 'driven' tag present?
            if d.get('driven', False):
                self.driven = ds[2]                     # ds[2] is the tag field of this data struct

        return dslist
    

    # Create element's solid-model data structure:
    #  [ [nx3 array of points for each section], [array of triangle nodes for each section], 
    #      [css color string for each section], [hover text for each section] ]
    def _make_el_sm_ds(self, d, el_idx):
        seclens = self._sec_lens(d, el_idx)             # Section lengths
        endpts = np.hstack([0.0,np.cumsum(seclens)])    # Section endpoints

        pts = []                                        # Create arrays of points and triangles
        tris = []
        nsides = self._getg(d,'nsides',9)               # #sides on cylindrical shapes
        color = self._getg(d,'color','lightsteelblue')  # Display color for each section
        if isinstance(color,str):                       # Could be a string or list of strings...
            color = [color]*len(seclens)
        diams = self._getg(d,'display_dia',self._getg(d,'tubing_dia',None))  # Element diameters: try 'display_dia', then 'tubing_dia'
        assert diams is not None,  f"Error: Element[{el_idx}] 'tubing_dia' or 'display_dia' must be specified"
        for i in range(len(seclens)):                   # For each section...
            
            poly = SolidObject.poly_xz(nsides, diams[i]/2)                          # Create polygon in xz plane
            s, t = SolidObject.extrude_y(poly, seclens[i], close_end=(False,True))  # Extrude to length of tubing section
            s = SolidObject.translate(s, (0,endpts[i],0))                           # Move along y axis

            pts.append(s)
            tris.append(t)

        text = self._getg(d,'text','')                      # Hover text for each section             
        suffix = self._getg(d,'suffix',['']*len(seclens))                    
        
        # Assemble the complete solid-model data structure
        return [pts, tris, color, [f'{text} {sfx}' for sfx in suffix]]



    #
    # Create a list of solid-model data structures from element properties and global properties
    #
    # Returns:  a list of solid-model data structs for elements, and possibly mirrored elements
    #
    def _create_sm_structs(self):
        dslist = []                                     # List of data structs to return
        for el_idx,d in enumerate(self.els):            # 'd' is the property dict for an element

            ds = self._make_el_sm_ds(d, el_idx)            # Create the nec data struct

            # Apply transforms from local properties dict
            rotate = d.get('rotate', None)              # Only local 'rotate' property
            mirror = self._getg(d,'display_mirror',False)   # Local or global
            translate = d.get('translate', None)        # Only local 'translate' property
            # Perform transforms on ds, possibly return mirrored element
            mirror_ds = self._rot_mir_trans(ds, rotate, mirror, translate)

            dslist.append(ds)                           # Append to element list
            if mirror_ds is not None:
                dslist.append(mirror_ds)

        # Add boom if specified
        boom = self._create_boom()
        if boom is not None:
            dslist.append(boom)

        # Apply global rotate, translate (there's no global 'mirror')
        rotate = self.properties.get('rotate', None)        # Only global 'rotate' property
        translate = self.properties.get('translate', None)  # Only global 'translate' property
        for ds in dslist:
            self._rot_mir_trans(ds, rotate=rotate, translate=translate)

        return dslist
    

    #
    # Create solid model of a (nonfunctional) yagi boom
    #
    # Returns:  solid-model data struct for boom, or None
    #
    def _create_boom(self):
        if not self.properties.get('boom', False):
            return None
        
        xpos = [d.get('translate',(0,0,0))[0] for d in self.els]        # Collect all element positions on y axis
        xmin = min(xpos) - self.properties.get('boomext', in2m(4))      # Ends of boom along x axis
        xmax = max(xpos) + self.properties.get('boomext', in2m(4))
        rect = SolidObject.rect_xz(self.properties.get('boomy', in2m(3)), self.properties.get('boomz', in2m(2)))      # Create rect in xz plane
        s, t = SolidObject.extrude_y(rect, xmax-xmin, close_end=(True,True)) 

        # Create sm data struct and rotate, translate into position
        ds = [[s], [t], [self.properties.get('boomcolor', 'steelblue')], [self.properties.get('text', '')]]
        self._rot_mir_trans(ds, rotate=[('z',90)], translate=(xmax,0,0))

        return ds





    # 
    # External interface: get NEC cards describing the antenna
    #
    # Returns:  'GW' cards for all antenna elements/sections as a single string
    #
    def get_nec_cards(self):

        dslist = self._create_nec_structs()             # Get list of nec data structs
        s = ''                                          # Output string

        for ds in dslist:                               # For each data struct in list...
            for idx in range(len(ds[0])):               # Each ds contains one or more segments
                x0,y0,z0 = ds[0][idx][0]                # First row of points: start of segment
                x1,y1,z1 = ds[0][idx][1]                # Second row of points: end of segment
                tag = ds[2]                             # Tag
                segs = ds[3][idx]                       # Segments in this wire
                rad = ds[1][idx]                        # Radius
                s += f'GW {tag} {segs} {x0} {y0} {z0} {x1} {y1} {z1} {rad}\n' 
                         
        return s
    
    #
    # Return tag of driven element
    #
    def get_driven_el_tag(self):
        return self.driven

 
    #
    # Return a list of one or more solid models for display
    #
    # Returns: [ [[point arrays], [triangle arrays], [css color strings], [hover text strings]] ]
    #
    def get_solid_models(self):
        return self._create_sm_structs()
  



In [126]:
#
# Simple solid object visualizer
#
#
class SolidObjVisualizer():

    def __init__(self, x,y,z, width=800, height=700) -> None:
        
        # Create some visual elements: sxy = 'ground level', sxz = mirroring plane (transparent)
        # To change colors, see https://plotly.com/python/builtin-colorscales/
        sxy = go.Surface(x=x, y=y, z=np.full((2,2),0), colorscale='Greens', surfacecolor=np.full((2,2),0.69), showscale=False,cmin=0,cmax=1, hoverinfo='skip')
        sxz = go.Mesh3d(x=(x[0],x[0],x[1],x[1]), y=(0,0,0,0), z=(z[0],z[1],z[1],z[0]), i=(0,0),j=(1,2),k=(2,3),color='lightpink', opacity=0.1, hoverinfo='skip')
    
        self.fig = go.Figure(data=[sxy, sxz])                        # Create the figure, with fixed visual elements

        # Set up appearance: size of viewer, range for each axis, etc.
        self.fig.update_layout(
            width=width, height=height, autosize=False,
            scene=dict(
                # 'eye' is your initial viewing angle
                camera=dict(up=dict(x=0,y=0,z=1),  eye=dict(x=1,y=.3,z=0.3)),
                # Relative display sizes of axes; this produces a square patch in xy plane, with z (elevation) half that length
                # aspectratio = dict( x=1, y=1, z=0.5 ),
                aspectratio = dict( x=1, y=1, z=1.0 ),
                aspectmode = 'manual',
                # Limits of each axis, in meters
                xaxis=dict(range=x, showbackground=False),
                yaxis=dict(range=y, showbackground=False),
                zaxis=dict(range=z, showbackground=False),
            ),
        )

    # Make visible
    def show(self):
        self.fig.show()

    # Add a solid object
    #   sobj        SolidObject instance
    def add(self, sobj, opacity=1.0):
        
        # Get list of solid-object data structs
        dslist = sobj.get_solid_models()

        for ds in dslist:
            for i in range(len(ds[0])):
                self.fig.add_mesh3d(i=ds[1][i][:,0], j=ds[1][i][:,1], k=ds[1][i][:,2], x=ds[0][i][:,0], y=ds[0][i][:,1], z=ds[0][i][:,2], 
                    color=ds[2][i], opacity=opacity, hovertemplate=ds[3][i]+'<extra></extra>')


    #

In [195]:
props = {'seg_per_m':5, 'nec_mirror':True, 'display_mirror':True, 'text':'Yagi', 'rotate':[('y',-30),('z',45)],'translate':(0,0,40)}
a = SolidAntenna(props, n_elem=3)

In [196]:
a[0] = {'len':2.5, 'translate':(-1,0,0), 'text':'R', 'suffix':['3/4', '5/8', '1/2']}
a[1] = {'len':2.25, 'translate':(0,0,0), 'text':'Dr', 'suffix':['3/4', '5/8', '1/2']}
a[2] = {'len':2.0, 'rotate':[('z',30)],'translate':(1.5,0,0), 'text':'D1', 'suffix':['3/4', '5/8', '1/2']}



In [197]:
v = SolidObjVisualizer(x=(-6,6), y=(-6,6), z=(34,46))
v.add(a)
v.show()
